# make a very simple LSTM model with keras

see how long this will take to run for 7t data with a not too complicated model and what kind of resources we are talking about - single run single subject..

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler
import pandas as pd
import nibabel as nb
import hrf_tools
#import hcp_utils as hcp
#from analysis import plot_59k_results

#import matplotlib.pyplot as plt


In [2]:
print(np.__version__)

1.19.5


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5561804427975421796
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10770692224
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5732604921821330734
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"
]


In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### load data

In [5]:
subject=100610
feature='rms'
n_movies=4
scaler = StandardScaler()

stim = ['tfMRI_MOVIE1_7T_AP','tfMRI_MOVIE2_7T_PA','tfMRI_MOVIE3_7T_PA','tfMRI_MOVIE4_7T_AP']
stim_feat = ['7T_MOVIE1_CC1_v2', '7T_MOVIE2_HO1_v2', '7T_MOVIE3_CC2_v2', '7T_MOVIE4_HO2_v2']
    
i=0
X = np.load(f'../sourcedata/data/HCP_7T_movie_FIX/features/{stim_feat[i]}_rms.npy')
X.shape#plt.plot(X[0:1000])

(9211, 1)

In [6]:
X  = np.load(f'../sourcedata/data/HCP_7T_movie_FIX/features/{stim_feat[i]}_as_embed.npy')

In [7]:
X.shape

(1918, 1024)

In [8]:
from scipy.signal import resample

In [9]:
X = resample(X, 921, axis=0) #resample to 1hz for now 

In [10]:
X_hrf = hrf_tools.apply_optimal_hrf_10hz(X,1)

In [11]:
#plt.plot(X[:,1])

In [12]:
img = nb.load('../sourcedata/data/parcellations/parcellated/sub100610_tfMRI_MOVIE1_7T_AP.ptseries.nii')
Y = img.get_fdata()
Y.shape

(921, 360)

In [13]:
img = nb.load('../sourcedata/data/parcellations/parcellated/sub100610_tfMRI_MOVIE2_7T_PA.ptseries.nii')
Y_test = img.get_fdata()
Y_test.shape

(918, 360)

In [14]:
i=1
X_test = np.load(f'../sourcedata/data/HCP_7T_movie_FIX/features/{stim_feat[i]}_rms.npy')
X_test = resample(X, 918, axis=0) #resample to 1hz for now 
X_test_hrf = hrf_tools.apply_optimal_hrf_10hz(X_test,1)
X_test.shape#plt.plot(X[0:1000])

(918, 1024)

In [15]:
# im_file = f'../sourcedata/data/HCP_7T_movie_FIX/brain/HCP_7T_movie_FIX/100610/MNINonLinear/Results/{stim[i]}/{stim[i]}_Atlas_1.6mm_MSMAll_hp2000_clean.dtseries.nii'
# img = nb.load(im_file)
# img_y = img.get_fdata()
# Y = img_y
#Y is time x brain
# Y.shape

In [16]:
#X is time x embed
X_hrf.shape

(921, 1024)

### design a simple(?) lstm model

In [17]:
# model = keras.Sequential(
#     [
#         layers.Dense(2, activation="relu", name="layer1"),
#         layers.Dense(3, activation="relu", name="layer2"),
#         layers.Dense(4, name="layer3"),
#     ]
# )
# # Call model on a test input
# x = tf.ones((3, 3))
# y = model(x)

In [18]:
# Batch size
batch_size = 64
time_steps = 10
input_units = 1024
input_shape = (time_steps,input_units)

model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
#model.add(layers.Embedding(input_dim=1024, output_dim=1024))
model.add(keras.Input(shape=input_shape))
# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(100))
#model.add(layers.UpSampling1D(size=10))
#model.add(layers.Dense(17049))
model.add(layers.Dense(200))
model.add(layers.Dense(300))

model.add(layers.Dense(360))



# Add a Dense layer with 10 units.
#model.add(layers.Dense(1721))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               450000    
_________________________________________________________________
dense (Dense)                (None, 200)               20200     
_________________________________________________________________
dense_1 (Dense)              (None, 300)               60300     
_________________________________________________________________
dense_2 (Dense)              (None, 360)               108360    
Total params: 638,860
Trainable params: 638,860
Non-trainable params: 0
_________________________________________________________________


In [19]:
# #dataset = tf.data.Dataset.from_tensor_slices((np.expand_dims(x_train,0), np.expand_dims(y_train,0)))
# dataset = tf.data.Dataset.from_tensor_slices((X_hrf, Y))

# dataset.element_spec

In [20]:
#https://keras.io/api/preprocessing/timeseries/
dataset = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_hrf, 
                                                     targets = Y, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)

In [21]:
dataset

<BatchDataset shapes: ((None, None, 1024), (None, 360)), types: (tf.float32, tf.float64)>

In [22]:


# # Buffer size to shuffle the dataset
# # (TF data is designed to work with possibly infinite sequences,
# # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# # it maintains a buffer in which it shuffles elements).
# BUFFER_SIZE = 10000

# dataset = (
#     dataset
#     .shuffle(BUFFER_SIZE)
#     .batch(batch_size, drop_remainder=True)
#     .prefetch(tf.data.experimental.AUTOTUNE))

# dataset

In [23]:
#simple_rnn = layers.SimpleRNN(18, return_sequences=True, return_state=True)

In [24]:
#whole_sequence_output, final_state = simple_rnn(np.expand_dims(x_train,0))

In [25]:
model.compile(
    loss=keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"),
    optimizer="adam",
    metrics=["accuracy", 'mean_squared_error'],
)

In [28]:
history = model.fit(dataset,epochs = 20)

Epoch 1/20
15/15 [==============================] - 0s 12ms/step - loss: 4718.5742 - accuracy: 0.9890 - mean_squared_error: 4718.5742
Epoch 2/20
15/15 [==============================] - 0s 12ms/step - loss: 4690.4814 - accuracy: 0.9890 - mean_squared_error: 4690.4814
Epoch 3/20
15/15 [==============================] - 0s 11ms/step - loss: 4712.2471 - accuracy: 0.9890 - mean_squared_error: 4712.2471
Epoch 4/20
15/15 [==============================] - 0s 11ms/step - loss: 4682.0088 - accuracy: 0.9890 - mean_squared_error: 4682.0088
Epoch 5/20
15/15 [==============================] - 0s 11ms/step - loss: 4709.7871 - accuracy: 0.9890 - mean_squared_error: 4709.7871
Epoch 6/20
15/15 [==============================] - 0s 11ms/step - loss: 4676.9424 - accuracy: 0.9890 - mean_squared_error: 4676.9424
Epoch 7/20
15/15 [==============================] - 0s 11ms/step - loss: 4709.8823 - accuracy: 0.9890 - mean_squared_error: 4709.8823
Epoch 8/20
15/15 [==============================] - 0s 11ms/st

In [ ]:
model.save('../outputs/model.pb')


In [ ]:
model = keras.models.load_model('../outputs/model.pb')

In [ ]:
dataset_test = tf.keras.preprocessing.timeseries_dataset_from_array(data = X_test_hrf, 
                                                     targets = Y_test, 
                                                    sequence_length=time_steps,
                                                     batch_size = batch_size)

In [ ]:
model.evaluate(dataset_test)